In [1]:
import joblib

cv, x_train, x_test, y_train, y_test = joblib.load('nsmc.pkl')

### 모형

In [2]:
import tensorflow as tf

In [3]:
model = tf.keras.models.Sequential()

In [6]:
model.add(  
    tf.keras.layers.Dense(
        16, 
        input_shape=(3239,),
        activation='relu',
        name='hidden'
    )
)

In [7]:
model.add(
    tf.keras.layers.Dense(
        1,  
        activation='sigmoid',
        name='output'
    )
)

In [8]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
hidden (Dense)               (None, 16)                51840     
_________________________________________________________________
output (Dense)               (None, 1)                 17        
Total params: 51,857
Trainable params: 51,857
Non-trainable params: 0
_________________________________________________________________


### 학습률

In [9]:
model.compile(optimizer=tf.keras.optimizers.Adam(lr=0.001),
             loss='binary_crossentropy',
             metrics=['accuracy'])

### 학습

In [11]:
model.fit(x_train.toarray(),y_train.values, epochs=100, validation_split=0.1,
         callbacks=[tf.keras.callbacks.EarlyStopping(monitor='val_accuracy')]) 

Train on 139924 samples, validate on 15548 samples
Epoch 1/100
139924/139924 [==============================] - 5s 37us/sample - loss: 0.3428 - accuracy: 0.8504 - val_loss: 0.3054 - val_accuracy: 0.8698
Epoch 2/100
139924/139924 [==============================] - 5s 36us/sample - loss: 0.2934 - accuracy: 0.8720 - val_loss: 0.3007 - val_accuracy: 0.8710
Epoch 3/100
139924/139924 [==============================] - 5s 37us/sample - loss: 0.2733 - accuracy: 0.8829 - val_loss: 0.3092 - val_accuracy: 0.8696


### 모형평가

In [12]:
model.evaluate(x_test.toarray(), y_test.values, verbose=0)

[0.31221665507777674, 0.867526]

# hyperopt

### 탐색공간 정의하기

In [13]:
from hyperopt import hp
from hyperopt.pyll.stochastic import sample

In [16]:
space = hp.choice('model',[
    {
        'layer': 'single',
        'lr': hp.loguniform('lr1',-7,2)  
    },
    {
        'layer': 'multi',
        'lr': hp.loguniform('lr2',-7,2),
        'units': hp.quniform('units',16,128,16),
        'activation': hp.choice('activation',['relu','tanh'])
    }
])

In [18]:
import joblib

cv, x_train, x_test, y_train, y_test = joblib.load('nsmc.pkl')

### 모형

In [19]:
import tensorflow as tf
from hyperopt import STATUS_OK
import uuid

In [21]:
def objective(h):
    model = tf.keras.models.Sequential()
    
    if h['layer'] == 'multi':
        model.add(tf.keras.layers.Dense(h['units'], activation=h['activation']))  # input_shape 자동지정댐.
    
    model.add(tf.keras.layers.Dense(1, activation='sigmoid'))
    
    model.compile(optimizer=tf.keras.optimizers.Adam(lr=h['lr']),
                 loss='binary_crossentropy',
                 metrics=['accuracy'])
    
    model.fit(
        x_train.toarray(),
        y_train.values,
        epochs=100,
        verbose=0,  
        callbacks=[tf.keras.callbacks.EarlyStopping(monitor='val_accuracy')]
    )
    
    loss, acc = model.evaluate(x_test.toarray(), y_test.values, verbose=0)
    
    filename = str(uuid.uuid4())
    model.save(filename)
    
    return{
        'loss': -acc,  
        'status': STATUS_OK,
        'filename': filename
    }

In [22]:
h = sample(space)

### 탐색

In [23]:
import warnings

In [24]:
warnings.simplefilter('ignore') 
tf.get_logger().setLevel('ERROR') 

In [25]:
objective(h)

{'loss': -0.7669805,
 'status': 'ok',
 'filename': '2a074037-9bfc-423d-a0b7-d85f4118a530'}

In [26]:
from hyperopt import fmin, rand, tpe, Trials

In [27]:
trials = Trials()

In [28]:
fmin(
    objective,
    space=space,
    algo=rand.suggest, 
    max_evals=3,
    trials=trials
)

100%|████████████████████████████████████████████████| 3/3 [20:43<00:00, 414.36s/trial, best loss: -0.8526808619499207]


{'lr1': 0.005418616500878853, 'model': 0}

In [30]:
best_model = tf.keras.models.load_model(trials.best_trial['result']['filename'])

In [31]:
best_model.evaluate(x_test.toarray(), y_test.values, verbose=0)

[0.3820252341719656, 0.85268086]